# Lecture 2.2: Tabular Data Pt.1


This lecture, we are going to investigate the data exploration powers of [pandas](https://pandas.pydata.org/).

**Learning goals:**
- Index and select from Dataframes
- Clean missing values
- Combine Dataframes
---

## 1. Indexing and selecting data

Since we will using `DataFrame`s _often_, it is important to get comfortable with common manipulation operations with pandas 🐼, such as indexing and selecting data.

### 1.1 `[]`

There are three main ways of selecting data in a `DataFrame`. The simplest is `[]`, i.e the standard python list notation. 


On a `DataFrame` object, column name(s) are passed as argument(s) to return the selected columns: 

    column_a = df['A']
    
On a `Series` object, an index label is used as argument to return the selected value:
    
    value_4 = s[4]
    
Let's try this out on a real dataset. The `top50.csv` file contains data on the spotify top songs of 2019. 🎵
We can load this conveniently with the `.read_csv()` function, one of many [i/o tools](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html) in pandas:

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('top50.csv', encoding = "latin")
df.head()

The `.head()` method gives us an overview of the first five rows. There are a lot of columns though... Let's just select the artist column:

In [ ]:
df['Artist.Name'].head()

Now we would like to see the song names too. This can be done with the `[]` operator by passing a list of column names. The double `[[` looks a bit goofy, but it works!

In [ ]:
df[['Track.Name', 'Artist.Name']].head()

What if we would like to index (add) data? The `[]` notation offers a neat way to add or replace columns in dataframes:

In [ ]:
df['Beats.Per.Minute.But.A.Bit.Faster'] = df['Beats.Per.Minute'] + 3
df['Beats.Per.Minute.But.A.Bit.Faster'].head()

ℹ️ Adding or replacing entire columns is fine, but careful with trying to set specific row values with the double `[][]` notation. This can lead to problems because the `[]` selector doesn't consistently return a view or a copy of the data. In those cases, pandas will raise a `SettingWithCopy` warning. More details [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-view-versus-copy).

## 1.2 `.loc[]`

`[]` is a convenient way to select data, but not the best way. Don't take my word for it, instead listen to the pandas [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html):

>The Python and NumPy indexing operators [] and attribute operator . provide quick and easy access to pandas data structures across a wide range of use cases. This makes interactive work intuitive, as there’s little new to learn if you already know how to deal with Python dictionaries and NumPy arrays. However, since the type of the data to be accessed isn’t known in advance, directly using standard operators has some optimization limits. For production code, we recommended that you take advantage of the optimized pandas data access methods exposed in this chapter.

One of these "optimized pandas data access methods" is `.loc[]`. It's a _label_ based selection method. It can be used with one or two arguments:

    df.loc[row_label]
    df.loc[row_label, column_label]

ℹ️ `.loc[]`'s first argument is the _row_ label. Remember that this is different from column names with `[]`! The reason `.loc[]` uses this "row first" notation, is because often we'll want to select on rows, and not on columns. 

In [ ]:
# select a row
df.loc[4]

In [ ]:
# select a cell
df.loc[4, 'Track.Name']

In [ ]:
#select a column
df.loc[:, 'Artist.Name'].head()

`.loc[]` is also cool because it supports label _slicing_. 😎

In [ ]:
# rows with label 2 to 5
df.loc[2:5]

In [ ]:
# columns with labels between Track.Name and Genre
df.loc[:, 'Track.Name':'Genre'].head()

Notice how the column slicing uses the _order_ of the column names to determine what's between `Track.Name` and `Genre`.

To select several rows/columns, we can also directly use label lists.

In [ ]:
# rows 4, 13, 44 and columns Genre & Energy
df.loc[[4, 12, 44], ['Genre', 'Energy']].head()

What if we want to index data? With `.loc[]`, we just have to assign data of correct shape to the selection:

In [ ]:
print('⏮ Before: ')
print(df.loc[2, ['Track.Name', 'Artist.Name']])
df.loc[2, ['Track.Name', 'Artist.Name']] = ['girlfriend (with Antisocial Flat)', 'Ariana Small']
print('\n⏭ After: ')
print(df.loc[2, ['Track.Name', 'Artist.Name']])

💪 A mistake seems to have slipped in our dataset... Using the method of your choice, write a function which sets the `Energy` of `ROSALÍA`'s big tune `Con Altura` to 9999. The method should return the `DataFrame` after having modified it. The unit test should pass!

In [ ]:
def fix_energy(df):
    # INSERT YOUR CODE HERE
    return df
    
def test_energy():
    fixed_df = fix_energy(df.copy())
    assert fixed_df.loc[44, 'Energy'] == 9999
    print('Success! 🎉')
    
test_energy()

## 1.3 `.iloc[]`

`.iloc[]` works just like `.loc[]`, except with _indices_ instead of _labels_. i.e It accepts the row and column positions as opposed to their names. Careful, it can get confusing when your index labels are numbers! It can be used with one or two arguments:

    df.iloc[row_index]
    df.iloc[row_index, column_index]

In [ ]:
# row at position 3
df.iloc[3]

In [ ]:
# every 3rd row between the first and 11th, and every column between the second and fith
df.iloc[0:10:3, 1:4]

To index values with `.iloc[]`, we also just have to assign data of correct shape to the selection:

In [ ]:
df.iloc[9, 1] = 'good gal'
df.iloc[9]

### 1.4 Boolean masks


Let's imagine that we are music snobs, and don't agree with the genre allocation of the songs. Everyone knows that pop with a bpm < 100 is actually called _slow pop_ 💁‍♂️. We _need_ to fix this travesty! 

Since there's 50 rows of data, we don't particularly want to go through the data manually. If only there was a way to do this easily with pandas...

In [ ]:
# update slow pop
df.loc[(df['Genre'] == 'pop') & (df['Beats.Per.Minute'] < 100), 'Genre'] = 'slow pop'
# show rows
df.loc[[3, 5, 7, 12, 37, 43, 49], :'Beats.Per.Minute']

Wow, all it took was one line! It's a complicated expression though, so let's break it down a bit.

In [ ]:
mask = (df['Genre'] == 'pop')
mask.head(10)

This is an example of an overloaded operator. The `==` here is applied _element wise_ to the `Series` returned by the `[]` selection. Therefore the one line expression is equivalent to the following:

    column_series = df['Genre']
    mask_series = (column == 'pop')
    
The result is a _mask_, i.e a list/array of booleans which indicates which elements of a matching list/array we wish to select. Here the mask is `True` only if the `Genre` is `pop`. What makes masks useful is that they share indices with their matching data list/array. So we can now use this mask to filter `pop` rows in `df`!

In [ ]:
df[df['Genre'] == 'pop']

Here we used a mask as argument to the `[]` operator, which returned all the rows where the mask was `True`. This creates an intuitive, terse, and powerful notation. Mastering boolean masks with pandas can greatly simplify your code!

Let's take another look at the original one liner:

    df.loc[(df['Genre'] == 'pop') & (df['Beats.Per.Minute'] < 100), 'Genre'] = 'slow pop'

We can notice the use of the `&` symbol. This is because boolean logic operators are themselves overloaded. This means that we can chain masks together. Say we wanted to view rows that qualify as `pop` _or_ were performed by `Katy Perry`: 

In [ ]:
df[(df['Genre'] == 'pop') | (df['Artist.Name'] == 'Katy Perry')]

Here we used two masks, chained together with an OR operator (`|`), and used as argument to the `[]` selection operator. The above line is equivalent to the following code:

    mask1 = (df['Genre'] == 'pop')
    mask2 = (df['Artist.Name'] == 'Katy Perry')
    mask1_or_mask2 = mask1 | mask2
    pop_or_katy_perry_df = df[mask1_or_mask2]
    
We can basically select anything! However, you might recall that in the complex one liner, we not only _selected_ data using chained boolean masks, but we also _replaced_ some of the selected values:

    df.loc[(df['Genre'] == 'pop') & (df['Beats.Per.Minute'] < 100), 'Genre'] = 'slow pop'
    
We've seen that updating specific rows is a bad idea using the `[]` notation, so we'll be using `.loc[]` instead here. The trick is that these masks can be fed as first argument of the `.loc[]` method, whilst the second argument can be used to select some columns. In our case, we'd like to select the `Genre` column of the filtered rows, since that's the column we wish to update to `slow pop`. So the complex one liner is equivalent to the following code:

    pop_mask = (df['Genre'] == 'pop')
    slow_bpm_mask = (df['Beats.Per.Minute'] < 100)
    pop_and_slow_bpm_mask = pop_mask & slow_bpm_mask
    cells_to_update = df.loc[pop_and_slow_bpm_mask, 'Genre']
    cells_to_update = 'slow pop'
    
🧠 There's a lot happening here, so take the time to understand how the code block above corresponds to the long one liner. Don't be worried if boolean masks feel like magic at first, they take a while to get used to! 🧙‍♀️
    
💪💪 Using a single line expression with boolean masks, update the `Genre` of the cells with `Loudness..dB..` >= -2.0 _OR_ `Liveness` > 40.0 as `annoying`. Your function below should return the modified `DataFrame`, and the unit test should pass!
  

    

In [ ]:
def replace_annoying(df):
    # INSERT YOUR CODE HERE
    return df

def test_annoying():
    new_df = replace_annoying(df.copy())
    n_replaced = new_df.loc[new_df['Genre'] == 'annoying', :].shape[0]
    assert n_replaced == 4
    print('Success! 🎉')
    
test_annoying()
    
    

## 2. Cleaning missing data

Missing values are common when exploring tabular or time series data. e.g missing records or temporary sensor failures. These can become a problem if used to train or test machine learning models. Knowing how to remove or fill in these missing values is therefore an essential skill for data scientists. Luckily, pandas offers many useful methods to make this process easier.

Let's take our spotify `DataFrame` and add a few random `None` values to imitate a faulty dataset:

In [ ]:
def add_missing_values(df, n):
    n_rows, n_columns = df.shape
    np.random.seed(1337)
    row_indices = np.random.randint(0, n_rows, n)
    column_indices = np.random.randint(0, n_columns, n)
    index_locations = zip(row_indices, column_indices)
    for iloc in index_locations:
        df.iloc[iloc] = None
    return df
        
df = pd.read_csv('top50.csv', encoding = "latin")
dirty_df = add_missing_values(df, 10)

We've added 10 missing values to a $50\times14$ matrix ... It's not going to be simple to spot them. We could try to make a boolean masks by checking if the values are null, but pandas makes it easy by supplying the mask directly with the `DataFrame.isna()` method:

In [ ]:
dirty_df.isna().head()

We can therefore use it to find the rows with a little help from the `.any()` method:

In [ ]:
# show rows with missing values
dirty_df.loc[df.isna().any(axis=1)]

🧠 Can you explain how the previous cell used boolean masks to return the rows with missing elements?

ℹ️ Here, missing values are represented as `NaN`. pandas actually supports several missing data types depending on the `dtype` of the `DataFrame`, and tries to take care of conversions in the background. Being aware of the differences can help when debugging, more details can be found in the [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#integer-dtypes-and-missing-data).

Now we see the extent of the damage to our data 😰. The sight of this corrupted dataset is unbearable and therefore we would like to clean the `DataFrame`. Since there is no way for us to guess what the values were, the best we can do to make downstream analysis smoother is to remove all the rows with `NaN` values.

We've learned how to set values using boolean masks, but again, pandas makes things easy for us:

In [ ]:
# drop the rows with missing values
clean_df = df.dropna()
# show rows with missing values
clean_df.loc[clean_df.isna().any(axis=1)]

The `.dropna()` method has magically removed all the dirty rows! More details about this api can be found [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#dropping-axis-labels-with-missing-data-dropna).

Sometimes, we don't want to remove missing data, but replace it with a default value instead. Let's take our `dirty_df` and replace all the `NaN` values in the `Danceability` column with a default value of 80:

In [ ]:
fill_df = dirty_df.copy()
fill_df['Danceability'] = fill_df['Danceability'].fillna(80.0)
fill_df.iloc[[8, 28, 40], [1, 2, 6]]

Groovy 🕺. The `.fillna()` method makes it easy to replace `NaN`s. pandas has versatile tools for missing data, e.g: interpolation of values, or replacing anything with anything really. More details can be found [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html). 

## 3. Combining data

Whether we have loaded datasets separately, or we want to aggregate different selections, we'll often have to combine `DataFrame` objects. There are three main methods available:

### 3.1 `df.append()`

Following the philosophy of keeping notation as close to python as possible, pandas `DataFrame` offers the same `.append()` method as lists.

Here, we select two row slices with `.iloc[]`, which we then combine together using `df.append()`.

In [ ]:
df = pd.read_csv('top50.csv', encoding = "latin")
df1 = df.iloc[:3, :3]
df2 = df.iloc[5:8, :3]
df3 = df1.append(df2)
df3

### 3.2 `pd.concat()`

`.concat()` offers more control and flexibility than `.append()`, mainly around column and index conflicts.

In [ ]:
df3 = pd.concat([df1, df2])
df3

### 3.3 `pd.merge()`

pandas also offers an api for database-style joins: `pd.merge()`. This function is complex and requires relational database basics, since it mirrors the SQL syntax used in join queries. We won't cover this api in this course, but keep in mind that it is useful because of its fast performance and high level of control. Detailed documentation can be found [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging).

## 4. Summary

Today, we learned more about **pandas**. We're now pros at **indexing** and **selecting** from `DataFrame`s, with the **`.loc[]`** and **`.iloc[]`** methods as well as **boolean masks**. We also know how to clean **missing values**. Finally, we saw how to combine `DataFrame`s.


# Resources

## Core Resources

The following four links are from the excellent official pandas documentation:
- [I/O tools](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html)
- [Indexing and selecting data](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html)
- [Merge, join, and concatenate](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)
- [Working with missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)

## Additional Resources

- [10 minutes to pandas](https://pandas.pydata.org/docs/getting_started/10min.html)  
Official introduction to the pandas library
- [Fast pandas](https://tomaugspurger.github.io/modern-4-performance)
